In [1]:
#Text generation using a character LSTM, specifically we want to
#generate new names as inspiration for those having a baby :) 
#Although this is for name generation, the code is general in the
#way that you can just send in any large text file (shakespear text, etc)
#and it will generate it.

In [2]:
import torch
import torch.nn as nn
import string
import random
import sys
import unidecode # ensure all names are in ascii format

c:\AI\NLP\MyPractise\PyTorchFundamental\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
# Get characters from string.printable
all_characters = string.printable
n_characters = len(all_characters)
n_characters

100

In [5]:
# Read large text file (Note can be any text file: not limited to just names)
file_data = unidecode.unidecode(open("data/names.txt").read())
len(file_data)

7303744

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embed = nn.Embedding(input_size, hidden_size)     # input_size: no of char ie 100, 
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        out = self.embed(x)
        out, (hidden, cell) = self.lstm(out.unsqueeze(1), (hidden, cell))
        out = self.fc(out.reshape(out.shape[0], -1))
        return out, (hidden, cell)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return hidden, cell

In [11]:
class Generator:
    def __init__(self):
        self.chunk_len = 250
        self.num_epochs = 5000
        self.batch_size = 1
        self.print_every = 50  # (50th epoch)
        self.hidden_size = 256
        self.num_layers = 2
        self.lr = 0.003

    def char_tensor(self, string):
        tensor = torch.zeros(len(string)).long()
        for c in range(len(string)):
            tensor[c] = all_characters.index(string[c])
        return tensor

    def get_random_batch(self):
        start_idx = random.randint(0, len(file_data) - self.chunk_len)
        end_idx = start_idx + self.chunk_len + 1
        text_str = file_data[start_idx:end_idx]
        text_input = torch.zeros(self.batch_size, self.chunk_len)
        text_target = torch.zeros(self.batch_size, self.chunk_len)

        for i in range(self.batch_size):
            text_input[i, :] = self.char_tensor(text_str[:-1])
            text_target[i, :] = self.char_tensor(text_str[1:])

        return text_input.long(), text_target.long()

    def generate(self, initial_str="A", predict_len=100, temperature=0.85):
        hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)
        initial_input = self.char_tensor(initial_str)
        predicted = initial_str

        for p in range(len(initial_str) - 1):
            _, (hidden, cell) = self.rnn(
                initial_input[p].view(1).to(device), hidden, cell
            )

        last_char = initial_input[-1]

        for p in range(predict_len):
            output, (hidden, cell) = self.rnn(
                last_char.view(1).to(device), hidden, cell
            )
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            predicted_char = all_characters[top_char]
            predicted += predicted_char
            last_char = self.char_tensor(predicted_char)

        return predicted

    # input_size, hidden_size, num_layers, output_size
    def train(self):
        self.rnn = RNN(
            n_characters, self.hidden_size, self.num_layers, n_characters
        ).to(device)

        optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        #writer = SummaryWriter(f"runs/names0")  # for tensorboard

        print("=> Starting training")

        for epoch in range(1, self.num_epochs + 1):
            inp, target = self.get_random_batch()
            hidden, cell = self.rnn.init_hidden(batch_size=self.batch_size)

            self.rnn.zero_grad()
            loss = 0
            inp = inp.to(device)
            target = target.to(device)

            for c in range(self.chunk_len):
                output, (hidden, cell) = self.rnn(inp[:, c], hidden, cell)
                loss += criterion(output, target[:, c])

            loss.backward()
            optimizer.step()
            loss = loss.item() / self.chunk_len

            if epoch % self.print_every == 0:
                print(f"Loss: {loss}")
                print(self.generate())

            #writer.add_scalar("Training loss", loss, global_step=epoch)

In [12]:
gennames = Generator()
gennames.train()

=> Starting training
Loss: 2.443293701171875
Anie
Arabrded
Nelylyn
olo
kay
Rilany
Pimeyne
iole
Lirer
Lenm
Marienioce
Larrio
Jiyn
Hreelind
Anean
Id
Loss: 2.234395263671875
Arnian
Kalra
Jarhian
Gamy
Hoyld
Malendha
Ccilenna
Sazley
Rallondd
Anna
Cabalde
Shory
Ery
Cecharie
Ane
Loss: 2.086257080078125
Alolin
Lonne
Loyke
Beannie
Rim
Alarie
Karle
Dareth
Pron
Lirigiel
Aina
Toray
Jary
Aydolla
Jonder
Wenne
Loss: 2.08497314453125
Al
Galda
Gergellie
Canger
Penrani
Dollory
Karit
Jamalren
Kerlyn
Pory
Shany
Juct
Fusel
Nex
Rarris
Arlo
Loss: 2.01928759765625
Astha
LaMa
Izaly
Maune
Codia
Jeshan
Ranie
Yean
yummen
Rascie
Ryla
Kever
Rosa
Tyler
Vion
Maris
Sarin
S
Loss: 2.049775146484375
Alynde
Shandi
Derdy
Jefger
Yedes
Marisa
Imevi
Sarr
Elrine
Rornie
Hoy
Doris
Neda
Dlida
Rheline
Kylda
B
Loss: 2.12375732421875
Anda
Mabila
Shannia
Kadain
Handid
Caydandal
Janes
Jochy
Juelma
Marick
Jhey
Magbeldia
Bryon
Ania
Jacka
Loss: 1.670894775390625
Anton
Dessa
Teann
Illy
Jotian
Styly
Jeandan
Frande
Jouian
Kady
Caxad
Leten
K